In [1]:
!nvidia-smi

Tue Feb  1 07:46:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 匯入函式庫

In [2]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm


# 匯入資料與前處理

In [3]:
diff_log_data = pd.read_csv('./rolling_linear_residue.csv', index_col=0)
diff_log_data = diff_log_data.astype('float64')
diff_log_data.index = pd.to_datetime(diff_log_data.index)

diff_log_data

,cad_rolling_linear_residue,aud_rolling_linear_residue,gbp_rolling_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000126,0.001068,-0.000024
2020-12-28,0.001416,-0.003410,-0.008017
2020-12-29,0.002184,0.003571,0.004029
2020-12-30,0.005244,0.010477,0.008856


# CPU/GPU、自定義資料集、模型、訓練函數

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
def SetSeed(myseed):
    # Python random module
    random.seed(myseed)
    # Numpy
    np.random.seed(myseed)
    # Torch
    torch.manual_seed(myseed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(myseed)
        torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [6]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, WindowSize):
        X = np.expand_dims(X, 1)
        self.X = X.astype(np.float64)
        self.WindowSize = WindowSize
        
    def __len__(self):
        return len(self.X) - self.WindowSize

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.WindowSize], self.X[idx+self.WindowSize])
        # return (X = [seqs, features], y)

In [7]:
class Self_Attention(nn.Module):
    def __init__(self, num_layers, hidden_size, length_input_sequence):
        super().__init__()
        self.num_layers = num_layers
        
        self.Input_First_HiddenLayer = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1)
        self.Query_Key_Value_1 = nn.ModuleList([nn.Linear(1, hidden_size), nn.Linear(1, hidden_size), nn.Linear(1, hidden_size)])

        self.Second_And_Following_HiddenLayer = nn.ModuleList([nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1) for i in range(num_layers - 1)])
        self.List_of_Query_Key_Value = nn.ModuleList([nn.ModuleList([nn.Linear(hidden_size, hidden_size) for q_k_v in range(3)]) for i in range(num_layers - 1)])

        self.OutputLayer = nn.Linear(hidden_size, 1)

        # generate mask
        mask = (torch.triu(torch.ones(length_input_sequence, length_input_sequence)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)
        self.register_buffer('mask', mask)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        Query1 = self.Query_Key_Value_1[0](input)
        Key1 = self.Query_Key_Value_1[1](input)
        Value1 = self.Query_Key_Value_1[2](input)
        hidden, _ = self.Input_First_HiddenLayer(Query1, Key1, Value1, attn_mask=self.mask)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        if self.num_layers > 1:
            for i, Second_And_Following_HiddenLayer in enumerate(self.Second_And_Following_HiddenLayer):
                Query = self.List_of_Query_Key_Value[i][0](hidden)
                Key = self.List_of_Query_Key_Value[i][1](hidden)
                Value = self.List_of_Query_Key_Value[i][2](hidden)
                hidden, _ = Second_And_Following_HiddenLayer(Query, Key, Value, attn_mask=self.mask)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [8]:
def train_under_config_and_evaluating_at_num_epochs_list(forex_data,
                                                         length_input_sequence,
                                                         num_epochs_list,
                                                         num_hidden_layers,
                                                         num_hidden_sizes,
                                                         batch_sizes,
                                                         device):
    '''
    forex_data,
    length_input_sequence,
    num_epochs_list,
    learning_rate,
    num_hidden_layers,
    num_hidden_sizes,
    batch_sizes,
    device
    '''
    # setseed
    SetSeed(9527)
    
    # dataset_train
    training_data = forex_data.loc['1981-01-01':'2008-12-31']
    training_dataset = TimeSeriesDataset(training_data, length_input_sequence)
    # dataset_valid
    validation_start_index = len(forex_data.loc['1981-01-01':'2008-12-31']) - length_input_sequence
    validation_end_index = len(forex_data.loc['1981-01-01':'2016-12-31'])
    validation_data = forex_data[validation_start_index:validation_end_index]
    validation_dataset = TimeSeriesDataset(validation_data, length_input_sequence)
    
    # dataloader_train
    training_dataloader = DataLoader(training_dataset, batch_size=batch_sizes, shuffle=True)
    # dataloader_valid
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_sizes, shuffle=False)
    
    # model
    model = Self_Attention(num_hidden_layers, num_hidden_sizes, length_input_sequence).double()
    # criterion & optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())
    
    # training & evaluating
    min_valid_loss_at_best_epoch = 100000
    min_valid_loss_epoch = 0
    for epoch in tqdm(range(num_epochs_list[-1])):
        # training
        model.to(device)
        model.train()
        for X, y in training_dataloader:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            ypred = model(X)
            loss = criterion(ypred, y)
            loss.backward()
            optimizer.step()
            
        # evaluating
        if (epoch + 1) in num_epochs_list:
            model.eval()
            valid_loss = 0
            len_valid = 0
            for X, y in  validation_dataloader:
                len_valid += len(X)
                X, y = X.to(device), y.to(device)
                with torch.no_grad():
                    ypred = model(X)
                    loss = criterion(ypred, y)
                valid_loss += loss.detach().cpu().item() * len(X)
            valid_loss = valid_loss / len_valid
            if valid_loss < min_valid_loss_at_best_epoch:
                min_valid_loss_at_best_epoch = valid_loss
                min_valid_loss_epoch = epoch + 1
    
    return min_valid_loss_epoch, min_valid_loss_at_best_epoch

In [9]:
def find_optimum_config_under_specific_input_length(forex_data,
                                                    length_input_sequence,
                                                    num_epochs_list,
                                                    num_hidden_layers_list,
                                                    num_hidden_sizes_list,
                                                    batch_sizes_list,
                                                    device=device):
    min_valid_loss = 100000
    min_valid_config_under_specific_input_length = ()
    for num_hidden_layers in num_hidden_layers_list:
        for num_hidden_sizes in num_hidden_sizes_list:
            for batch_sizes in batch_sizes_list:
                print('\nTraining under config:', (num_hidden_layers, num_hidden_sizes, batch_sizes))
                min_valid_loss_epoch, min_valid_loss_at_best_epoch = train_under_config_and_evaluating_at_num_epochs_list(forex_data,
                                                                                                                          length_input_sequence,
                                                                                                                          num_epochs_list,
                                                                                                                          num_hidden_layers,
                                                                                                                          num_hidden_sizes,
                                                                                                                          batch_sizes,
                                                                                                                          device)
                if min_valid_loss_at_best_epoch < min_valid_loss:
                    min_valid_loss = min_valid_loss_at_best_epoch
                    min_valid_config_under_specific_input_length = (num_hidden_layers, num_hidden_sizes, batch_sizes, min_valid_loss_epoch)

                    print('\nvalid_loss improve to',
                            min_valid_loss,
                            'under config:',
                            min_valid_config_under_specific_input_length,
                            '(num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)')

    print('\nmin valid loss config under specific input length',
          length_input_sequence,
          'is:',
          min_valid_config_under_specific_input_length,
          '(num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)',
          'and the valid loss is:',
          min_valid_loss)
    return min_valid_config_under_specific_input_length, min_valid_loss

# cad

In [10]:
cad_data = diff_log_data.iloc[:,0]
print(cad_data.loc['1981-01-01':'2008-12-31'])
print(cad_data.loc['2009-01-01':'2016-12-31'])
print(cad_data.loc['2017-01-01':'2020-12-31'])

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2008-12-25   -0.001197
2008-12-26   -0.004123
2008-12-29   -0.000227
2008-12-30    0.003186
2008-12-31   -0.002053
Name: cad_rolling_linear_residue, Length: 7304, dtype: float64
2009-01-01   -0.000714
2009-01-02    0.008280
2009-01-05    0.017649
2009-01-06    0.004052
2009-01-07   -0.008306
                ...   
2016-12-26    0.002181
2016-12-27   -0.004858
2016-12-28    0.001239
2016-12-29    0.004212
2016-12-30    0.004732
Name: cad_rolling_linear_residue, Length: 2087, dtype: float64
2017-01-02   -0.000251
2017-01-03    0.001227
2017-01-04    0.009506
2017-01-05    0.005720
2017-01-06   -0.000910
                ...   
2020-12-25   -0.000126
2020-12-28    0.001416
2020-12-29    0.002184
2020-12-30    0.005244
2020-12-31    0.001403
Name: cad_rolling_linear_residue, Length: 1044, dtype: float64


In [11]:
# random walk mse(validation)
se = cad_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

3.594535629597425e-05

In [12]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [13]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:38<00:00,  1.28s/it]



valid_loss improve to 3.619028762416368e-05 under config: (1, 25, 16, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:20<00:00,  1.46it/s]



valid_loss improve to 3.589746169273024e-05 under config: (1, 25, 32, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [02:46<00:00,  5.54s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [02:12<00:00,  4.43s/it]


min valid loss config under specific input length 5 is: (1, 200, 256, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.5865749530947545e-05


((1, 200, 256, 15), 3.5865749530947545e-05)

In [14]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [15]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:38<00:00,  1.29s/it]



valid_loss improve to 3.603951726915599e-05 under config: (1, 25, 16, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:19<00:00,  1.50it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [04:23<00:00,  8.79s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [04:00<00:00,  8.00s/it]


min valid loss config under specific input length 10 is: (1, 400, 256, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.590013079817501e-05


((1, 400, 256, 15), 3.590013079817501e-05)

In [16]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [17]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:38<00:00,  1.30s/it]



valid_loss improve to 3.6284095836188113e-05 under config: (1, 25, 16, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:20<00:00,  1.50it/s]



valid_loss improve to 3.5924713039320544e-05 under config: (1, 25, 32, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [07:59<00:00, 15.99s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [07:36<00:00, 15.21s/it]


min valid loss config under specific input length 20 is: (1, 25, 64, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.5910990220324696e-05


((1, 25, 64, 5), 3.5910990220324696e-05)

In [18]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [19]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:39<00:00,  1.32s/it]



valid_loss improve to 3.6802929865190886e-05 under config: (1, 25, 16, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:20<00:00,  1.45it/s]



valid_loss improve to 3.593949660767751e-05 under config: (1, 25, 32, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [22:12<00:00, 44.42s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [21:53<00:00, 43.79s/it]


min valid loss config under specific input length 60 is: (1, 400, 128, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.591432346448313e-05


((1, 400, 128, 5), 3.591432346448313e-05)

# aud

In [10]:
aud_data = diff_log_data.iloc[:,1]
print(aud_data.loc['1981-01-01':'2008-12-31'])
print(aud_data.loc['2009-01-01':'2016-12-31'])
print(aud_data.loc['2017-01-01':'2020-12-31'])

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2008-12-25   -0.000674
2008-12-26    0.001521
2008-12-29    0.002103
2008-12-30    0.009210
2008-12-31    0.016125
Name: aud_rolling_linear_residue, Length: 7304, dtype: float64
2009-01-01    0.003893
2009-01-02    0.008246
2009-01-05    0.008877
2009-01-06    0.008109
2009-01-07   -0.014846
                ...   
2016-12-26    0.002285
2016-12-27   -0.000916
2016-12-28   -0.001056
2016-12-29    0.005893
2016-12-30   -0.001467
Name: aud_rolling_linear_residue, Length: 2087, dtype: float64
2017-01-02   -0.003278
2017-01-03    0.004918
2017-01-04    0.008884
2017-01-05    0.007581
2017-01-06   -0.004999
                ...   
2020-12-25    0.001068
2020-12-28   -0.003410
2020-12-29    0.003571
2020-12-30    0.010477
2020-12-31    0.001053
Name: aud_rolling_linear_residue, Length: 1044, dtype: float64


In [21]:
# random walk mse(validation)
se = aud_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

6.228930968484775e-05

In [22]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [23]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:39<00:00,  1.31s/it]



valid_loss improve to 6.225207223336015e-05 under config: (1, 25, 16, 25) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:20<00:00,  1.49it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [02:46<00:00,  5.54s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [02:12<00:00,  4.43s/it]


min valid loss config under specific input length 5 is: (2, 400, 16, 25) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 6.217447132295218e-05


((2, 400, 16, 25), 6.217447132295218e-05)

In [24]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [25]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:38<00:00,  1.30s/it]



valid_loss improve to 6.230954282255556e-05 under config: (1, 25, 16, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:20<00:00,  1.49it/s]



valid_loss improve to 6.225874964955444e-05 under config: (1, 25, 32, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [04:23<00:00,  8.80s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [04:00<00:00,  8.02s/it]


min valid loss config under specific input length 10 is: (1, 800, 16, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 6.215137756836581e-05


((1, 800, 16, 30), 6.215137756836581e-05)

In [26]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [27]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:39<00:00,  1.30s/it]



valid_loss improve to 6.260105675194014e-05 under config: (1, 25, 16, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:20<00:00,  1.45it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [07:59<00:00, 15.97s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [07:37<00:00, 15.26s/it]


min valid loss config under specific input length 20 is: (1, 200, 256, 5) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 6.225102739292881e-05


((1, 200, 256, 5), 6.225102739292881e-05)

In [11]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [12]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:37<00:00,  1.25s/it]



valid_loss improve to 6.2631760270393e-05 under config: (1, 25, 16, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:19<00:00,  1.53it/s]



valid_loss improve to 6.230545899291561e-05 under config: (1, 25, 32, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [22:20<00:00, 44.69s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [21:59<00:00, 43.97s/it]


min valid loss config under specific input length 60 is: (1, 400, 32, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 6.221917948346872e-05


((1, 400, 32, 15), 6.221917948346872e-05)

# gbp

In [13]:
gbp_data = diff_log_data.iloc[:,2]
print(gbp_data.loc['1981-01-01':'2008-12-31'])
print(gbp_data.loc['2009-01-01':'2016-12-31'])
print(gbp_data.loc['2017-01-01':'2020-12-31'])

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2008-12-25    0.000663
2008-12-26   -0.010867
2008-12-29   -0.012544
2008-12-30    0.001684
2008-12-31    0.012717
Name: gbp_rolling_linear_residue, Length: 7304, dtype: float64
2009-01-01    0.005450
2009-01-02   -0.009023
2009-01-05    0.010761
2009-01-06    0.014373
2009-01-07    0.011438
                ...   
2016-12-26    0.000071
2016-12-27   -0.000828
2016-12-28   -0.003576
2016-12-29    0.003050
2016-12-30    0.006395
Name: gbp_rolling_linear_residue, Length: 2087, dtype: float64
2017-01-02   -0.005184
2017-01-03   -0.003108
2017-01-04    0.007186
2017-01-05    0.007595
2017-01-06   -0.010878
                ...   
2020-12-25   -0.000024
2020-12-28   -0.008017
2020-12-29    0.004029
2020-12-30    0.008856
2020-12-31    0.002988
Name: gbp_rolling_linear_residue, Length: 1044, dtype: float64


In [14]:
# random walk mse(validation)
se = gbp_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

3.661160280198625e-05

In [15]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [16]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:37<00:00,  1.24s/it]



valid_loss improve to 3.899777768027485e-05 under config: (1, 25, 16, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:19<00:00,  1.55it/s]



valid_loss improve to 3.6619675942162834e-05 under config: (1, 25, 32, 25) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [02:46<00:00,  5.55s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [02:13<00:00,  4.46s/it]


min valid loss config under specific input length 5 is: (1, 200, 256, 20) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.655249105749843e-05


((1, 200, 256, 20), 3.655249105749843e-05)

In [17]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [18]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:38<00:00,  1.28s/it]



valid_loss improve to 3.660532126869439e-05 under config: (1, 25, 16, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:20<00:00,  1.49it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [04:24<00:00,  8.83s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [04:01<00:00,  8.06s/it]


min valid loss config under specific input length 10 is: (1, 400, 128, 30) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.6587514150219674e-05


((1, 400, 128, 30), 3.6587514150219674e-05)

In [19]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [20]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:37<00:00,  1.25s/it]



valid_loss improve to 3.668578654667644e-05 under config: (1, 25, 16, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:19<00:00,  1.52it/s]



valid_loss improve to 3.6616527857254105e-05 under config: (1, 25, 32, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [08:03<00:00, 16.11s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [07:40<00:00, 15.34s/it]


min valid loss config under specific input length 20 is: (1, 200, 128, 15) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.659983581314204e-05


((1, 200, 128, 15), 3.659983581314204e-05)

In [21]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [22]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )


Training under config: (1, 25, 16)


100%|██████████| 30/30 [00:38<00:00,  1.27s/it]



valid_loss improve to 3.668671252692954e-05 under config: (1, 25, 16, 25) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)

Training under config: (1, 25, 32)


100%|██████████| 30/30 [00:19<00:00,  1.50it/s]



Training under config: (1, 25, 64)


...

100%|██████████| 30/30 [22:20<00:00, 44.68s/it]



Training under config: (4, 1600, 256)


100%|██████████| 30/30 [21:53<00:00, 43.77s/it]


min valid loss config under specific input length 60 is: (1, 200, 256, 10) (num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs) and the valid loss is: 3.6605847939025465e-05


((1, 200, 256, 10), 3.6605847939025465e-05)